In [1]:
import torch
import evaluate
from rich import print, print_json
from shinkai.core.expirement import Expirement, ExpirementMetrics
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import DatasetDict 
from shinkai.constants import CACHE_DIR
from shinkai.core.loader import load_ift
from tqdm import tqdm

In [2]:
ds = load_ift(testSize=0.005)
print(ds)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1090
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 6
    })
})

In [3]:
device = "mps" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct", cache_dir="../" + CACHE_DIR).to(device)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct", cache_dir="../" + CACHE_DIR)

model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb)

In [4]:
ds_test = ds["test"]

In [ ]:
promt = """
You are a Swift documentation assistant.

Your task is to add precise and complete Swift-style documentation comments (///) to the function below. Follow Apple's Xcode documentation conventions.

The documentation must include:
- A one-sentence summary of the function’s purpose.
- A "- Parameters:" section explaining each parameter clearly, including its type and role.
- A "- Returns:" section that describes the return value (if the function returns something), with its type.

Guidelines:
- Use concise, technical English appropriate for developer-facing API docs.
- Include type hints in explanations (e.g., “an integer”, “a string array”).
- If the function is generic (<T>), explain the generic behavior clearly.
- Do not include any commentary or explanation outside the code block.
- The output must be a single Swift code block containing only the function and its documentation.

```swift
func median(of numbers: [Double]) -> Double? {
    guard !numbers.isEmpty else { return nil }
    let sorted = numbers.sorted()
    let mid = sorted.count / 2
    return sorted.count % 2 == 0
        ? (sorted[mid - 1] + sorted[mid]) / 2
        : sorted[mid]
}
```
"""

print(model.generation_config)
inputs = tokenizer(promt, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    max_new_tokens = 200,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,   # чтобы декодер не ругался
    do_sample=True,
    temperature=0.5,
    top_p=0.95      # запрет на повтор 4-грамм
)
print(outputs)

In [ ]:
result = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(result)

In [ ]:
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    return_dict_in_generate=True,
    output_scores = False
)

# Получаем только сгенерированную часть
generated_tokens = outputs.sequences[0][inputs.input_ids.shape[1]:]
result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(result)

### Metrics Test

In [ ]:
# promt = """
# You are a senior Swift developer.
# Given a Swift function, generate a short, plain-language summary as a documentation comment using ///.
# Keep the comment concise — just 1–2 sentences describing what the function does.
# Return only the documentation and the original function.
# It is very important that you only provide the final output without any additional comments or remarks
# func median(of numbers: [Double]) -> Double? {
#     guard !numbers.isEmpty else { return nil }
#     let sorted = numbers.sorted()
#     let mid = sorted.count / 2
#     return sorted.count % 2 == 0
#         ? (sorted[mid - 1] + sorted[mid]) / 2
#         : sorted[mid]
# }
# """

# Лучшая версия promts для deepseek-coder
promt = """
You are documenting a Swift codebase. Given a function and a short description of where and how it is used, write a documentation comment that helps future developers understand its purpose.
Use ///-style Swift comments.
Include parameter and return descriptions, and emphasize intent over implementation.
Return only the comment and the function, no extras, notes, no repeat output.

func randomElementFromStringArray(_ array: [String]) -> String? {
    return array.randomElement()
} 
"""

print(model.generation_config)
inputs = tokenizer(promt, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    max_new_tokens = 256,
    do_sample=True
)

print(outputs)

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

tensor([[  198,  2610,   525, 66971,   264, 23670,  2038,  3152,    13, 16246,
           264,   729,   323,   264,  2805,  4008,   315,  1380,   323,  1246,
           432,   374,  1483,    11,  3270,   264,  9705,  3980,   429,  8609,
          3853, 13402,  3535,  1181,  7428,   624, 10253,  1048,    12,  3528,
         23670,  6042,   624, 22283,  5733,   323,   470, 27787,    11,   323,
         45932,  7385,   916,  8129,   624,  5598,  1172,   279,  3980,   323,
           279,   729,    11,   902, 36275,    11,  8388,    11,   902, 13153,
          2550,   624,  5152,   510,  2830,  4194,  1691, 44491,  1857,  2490,
          1334,    25,   508,   703,  2467,  1464,   923,    30,   341,   262,
           470,  1334,  7829,  1691,   741,    92,   715,  5009,   510,  1986,
           729,  4675,   264,  4194,  2392,   504,   264,  2661,  1334,   315,
          9069,    11,   476,  2092,   421,   279,  1334,   374,  4287,   624,
           785,   729,   374,  1483,   304,   279,  1845,   594,  6200, 12218,
           311,  7807,   264,  4194,   914,   504,   264,  1140,   315,  9069,
           624,   785,   729,   374,  2598,   304,   279,  1887,   729,   315,
           279,  1845,   311,   633,   264,  4194,   914,   369,  3037,   382,
           785,  3980,  1265,   387,   304,   279,  1352,   315,   510,  2575,
           366,  1688, 11494,  2235,    27, 16181, 10604,    29,   366, 16181,
         11494,    29, 32574,  1464,   366,   689, 10604,   397,  2575,   256,
           366,  4684,   397,  2575,   257,   366, 16181, 52547,    81,   804,
           397,  2575,   981,   366, 16181, 52547,    81,   804,   397,  2575,
           257,   690, 16181, 52547,    81,   804,   397,  2575,   257,   366,
           689, 52547,    81,   804,   397,  2575,   981,   366,   689, 52547,
            81,   804,   397,  2575,   256,   690,  4684,   397,  2575,   690,
          1688, 11494,   397,   785,   729,   374,  2598,   304,   279,  1887,
           729,   315,   279,  1845,   311,   633,   264,  4194,   914,   369,
          3037,   624,   785,   729,   374,  1483,   304,   279,  1845,   594,
          6200, 12218,   311,  7807,   264,  4194,   914,   504,   264,  1140,
           315,  9069,   624,   785,   729,  4675,   264,  4194,  2392,   504,
           264,  2661,  1334,   315,  9069,    11,   476,  2092,   421,   279,
          1334,   374,  4287,   624,   785,   729,   374,  1483,   304,   279,
          1845,   594,  6200, 12218,   311,  7807,   264,  4194,   914,   504,
           264,  1140,   315,  9069,   624,   785,   729,   374,  1483,   304,
           279,  1887,   729,   315,   279,  1845,   311,   633,   264,  4194,
           914,   369,  3037,   624,   785,   729,  4675,   264,  4194,  2392,
           504,   264,  2661]])

In [10]:
# Получаем длины исходных входов
input_lengths = inputs['input_ids'].shape[1]
# Убираем токены prompt'а
generated_tokens = outputs[:, input_lengths:]
# Декодируем только новые токены
predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

for predict in predictions:
    print("Predict: \n", predict)

Predict: 
 Description:
This function returns a random element from a given array of strings, or nil if the array is empty.
The function is used in the App's core logic to fetch a random string from a list of strings.
The function is called in the main function of the App to get a random string for display.

The comment should be in the form of:
/// <function-name>(<parameter-type> <parameter-name>...) -> <return-type>
///   <description>
///     <parameter-descrations>
///       <parameter-descrations>
///     </parameter-descrations>
///     <return-descrations>
///       <return-descrations>
///   </description>
/// </function-name>
The function is called in the main function of the App to get a random string for display.
The function is used in the App's core logic to fetch a random string from a list of strings.
The function returns a random element from a given array of strings, or nil if the array is empty.
The function is used in the App's core logic to fetch a random string from a list of strings.
The function is used in the main function of the App to get a random string for display.
The function returns a random element from a given

In [46]:
print(outputs.shape)

torch.Size([1, 163])

In [47]:
predictions = []

for input in tqdm(ds_test["input"]):
    promt = f"""You are documenting a Swift codebase.
Given a function and a short description of where and how it is used, write a documentation comment that helps future developers understand its purpose.
Use ///-style Swift comments.
Include parameter and return descriptions, and emphasize intent over implementation.
Return only the comment and the function.
{input}
    """
    inputs = tokenizer(promt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens = 200,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,   # чтобы декодер не ругался
        do_sample=True,
        temperature=0.7,
        top_p=0.95)
    input_lengths = inputs.input_ids.shape[1]
    generated_tokens = outputs[:, input_lengths:]
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    predictions.append(result[0])

100%|██████████| 6/6 [02:37<00:00, 26.27s/it]


In [50]:
for predict in predictions:
    print("[bold green] Predict [/ bold green]")
    print(predict)

 Predict 

/**
 Returns a random element from an array of strings.
 @param array - An array of strings.
 @returns A random element from the array.
 */
func randomElementFromStringArray(_ array: [String]) -> String? {
    return array.randomElement()
}

 Predict 

// Returns the square root of a number.
//
// - number: The number to take the square root of.
// - returns: The square root of `number`.
func squareRoot(of number: Double) -> Double {
    return sqrt(number)
}

 Predict 

...

/////
// Checks if a given email address is valid.
//
// - Parameter email: The email address to check.
// - Returns: `true` if the email address is valid, `false` otherwise.
func isValidEmail(email: String) -> Bool {
    let pattern = "^[A-Z0-9._%+-]+@[A-Z0-9.-]+\\.[A-Z]{2,}$"
    let regex = try? NSRegularExpression(pattern: pattern, options: .caseInsensitive)
    return regex?.firstMatch(in: email, options: [], range: NSRange(location: 0, length: email.utf16.count)) != nil
}

 Predict 

// /// Adds the digits of a number.
// /// - Parameter number: The number to add the digits of.
// /// - Returns: The sum of the digits of number.
// func sumOfDigits(_ number: Int) -> Int {
//     return String(number).compactMap { Int(String($0)) }.reduce(0, +)
// }

func sumOfDigits(_ number: Int) -> Int {
    return String(number).compactMap { Int(String($0)) }.reduce(0, +)
}

// /// Adds the digits of a number.
// /// - Parameter number: The number to add the digits of.
// /// - Returns: The sum of the digits of number.
// func sumOfDigits(_ number: Int) -> Int {
//     return String

 Predict 

// Counts the number of unique values in an array of T.
//
// - Parameters:
//   - in array: The array to check for unique values.
//
// - Returns:
//   - The number of unique values in the array.
func countUnique(in array: [Int]) -> Int {
    return Set(array).count
}

// Counts the number of unique values in an array of T.
//
// - Parameters:
//   - in array: The array to check for unique values.
//
// - Returns:
//   - The number of unique values in the array.
func countUnique(in array: [String]) -> Int {
    return Set(array).count
}


// Counts the number of unique values in an array of T.
//
// - Parameters:
//   - in array

In [53]:
references = ds_test["output"]

print(references)

[
    '/// Генерирует случайный элемент из массива строк.\n/// - Parameter array: Массив строк.\n/// - Returns: 
Случайный элемент из массива.\nfunc randomElementFromStringArray(_ array: [String]) -> String? {\n    return 
array.randomElement()\n}',
    '/// Возвращает число, которое является квадратным корнем из заданного числа.\n/// - Parameter number: 
Число.\n/// - Returns: Квадратный корень числа.\nfunc squareRoot(of number: Double) -> Double {\n    return 
sqrt(number)\n}',
    '/// Проверяет, является ли массив палиндромом.\n/// - Parameter array: Массив элементов.\n/// - Returns: true,
если массив читается одинаково в обе стороны.\nfunc isPalindrome<T: Equatable>(_ array: [T]) -> Bool {\n    return 
array == array.reversed()\n}',
    '/// Проверяет, является ли строка допустимым email.\n/// - Parameter email: Строка для проверки.\n/// - 
Returns: true, если строка похожа на email.\nfunc isValidEmail(_ email: String) -> Bool {\n    let pattern = 
"^[A-Z0-9._%+-]+@[A-Z0-9.-]+\\\\.[A-Z]{2,}$"\n    let regex = try? NSRegularExpression(pattern: pattern, options: 
.caseInsensitive)\n    return regex?.firstMatch(in: email, options: [], range: NSRange(location: 0, length: 
email.utf16.count)) != nil\n}',
    '/// Считает сумму цифр числа.\n/// - Parameter number: Число.\n/// - Returns: Сумма цифр.\nfunc sumOfDigits(_ 
number: Int) -> Int {\n    return String(number).compactMap { Int(String($0)) }.reduce(0, +)\n}',
    '/// Возвращает количество уникальных элементов в массиве.\n/// - Parameter array: Входной массив.\n/// - 
Returns: Количество уникальных элементов.\nfunc countUnique<T: Hashable>(in array: [T]) -> Int {\n    return 
Set(array).count\n}'
]

In [54]:
rouge = evaluate.load("rouge")

In [55]:
# Вычисление метрик
results = rouge.compute(predictions=predictions, references=references)

In [56]:
# Вывод результатов
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.4927

rouge2: 0.3798

rougeL: 0.4832

rougeLsum: 0.4888

In [57]:
print(results)

{
    'rouge1': np.float64(0.4926743815794188),
    'rouge2': np.float64(0.37982149208674515),
    'rougeL': np.float64(0.48323344549967384),
    'rougeLsum': np.float64(0.48878264347473044)
}

In [59]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

{
    'bleu': 0.3932765237048014,
    'precisions': [0.47067039106145253, 0.4056338028169014, 0.3721590909090909, 0.336676217765043],
    'brevity_penalty': 1.0,
    'length_ratio': 1.6807511737089202,
    'translation_length': 716,
    'reference_length': 426
}

##### Big test set metrics

In [8]:
# BIG Test set
ds = load_ift(testSize=0.05)
print(ds)

device = "mps" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", cache_dir="../" + CACHE_DIR).to(device)
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", cache_dir="../" + CACHE_DIR)

model.eval()

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1041
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 55
    })
})

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-06)
    (rotary_emb)

In [ ]:
predictions = []

for input in tqdm(ds["test"]["input"]):
    promt = f"""You are documenting a Swift codebase.
Given a function and a short description of where and how it is used, write a documentation comment that helps future developers understand its purpose.
Use ///-style Swift comments.
Include parameter and return descriptions, and emphasize intent over implementation.
Return only the comment and the function.
{input}
    """
    inputs = tokenizer(promt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens = 200,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,   # чтобы декодер не ругался
        do_sample=True,
        temperature=0.7,
        top_p=0.95)
    input_lengths = inputs.input_ids.shape[1]
    generated_tokens = outputs[:, input_lengths:]
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    predictions.append(result[0])

100%|██████████| 55/55 [27:00<00:00, 29.46s/it]


In [15]:
references=ds["test"]["input"]

In [16]:
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=predictions, references=references)
print(results)

{
    'rouge1': np.float64(0.3907492461218042),
    'rouge2': np.float64(0.3445763016921688),
    'rougeL': np.float64(0.38462004211065065),
    'rougeLsum': np.float64(0.3905372102829945)
}

In [17]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

{
    'bleu': 0.26520786081010966,
    'precisions': [0.2920277296360485, 0.2661231620323191, 0.25579688875843853, 0.24885338067761503],
    'brevity_penalty': 1.0,
    'length_ratio': 2.8766098878271706,
    'translation_length': 6924,
    'reference_length': 2407
}

## Qwen Inference

In [ ]:
### Qwen 3 Inference

insert_function = """
func randomElementFromStringArray(_ array: [String]) -> String? {
    return array.randomElement()
} 
"""

# Лучшая версия promts для deepseek-coder
prompt = f"""
Write a Swift-style documentation comment (`///`) for the function below. The comment must describe the function’s purpose, parameters, and return value, using Apple's documentation conventions.

Only return the documentation comment and the function itself. **Do not use any Markdown formatting**, such as ``` or code blocks. Just plain Swift code with comments.

Function:

{insert_function}
"""

messages = [
    { "role": "user", "content": prompt }
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)

print(model.generation_config)

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    max_new_tokens = 256,
    do_sample=True
)

# Получаем длины исходных входов
input_lengths = inputs['input_ids'].shape[1]
# Убираем токены prompt'а
generated_tokens = outputs[:, input_lengths:]
# Декодируем только новые токены
predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

for predict in predictions:
    print("Predict: \n", predict)

print(outputs)

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

Predict: 
 /// Returns a randomly selected element from the given array of strings, or nil if the array is empty.
///
/// - Parameters:
///   - array: A non-nil array of strings.
/// - Returns: A randomly selected string, or nil if the array is empty.
///
/// @param array A non-nil array of strings.
/// @return A randomly selected string, or nil if the array is empty.
func randomElementFromStringArray(_ array: [String]) -> String? {
    return array.randomElement()
}

tensor([[151644,    872,    271,   7985,    264,  23670,  11297,   9705,   3980,
          28654,   2575,  32881,    369,    279,    729,   3685,     13,    576,
           3980,   1969,   7512,    279,    729,    748,   7428,     11,   5029,
             11,    323,    470,    897,     11,   1667,   8162,    594,   9705,
          44493,    382,   7308,    470,    279,   9705,   3980,    323,    279,
            729,   5086,     13,   3070,   5404,    537,    990,    894,  73192,
          36566,  97219,   1741,    438,  54275,    476,   2038,  10010,     13,
           4599,  14396,  23670,   2038,    448,   6042,    382,   5152,  24391,
           2830,   4194,   1691,  44491,   1857,   2490,   1334,     25,    508,
            703,   2467,   1464,    923,     30,    341,    262,    470,   1334,
           7829,   1691,    741,     92,  14731, 151645,    198, 151644,  77091,
            198, 151667,    271, 151668,    271,   2575,   5184,    264,  26618,
           4091,   2392,    504,    279,   2661,   1334,    315,   9069,     11,
            476,   2092,    421,    279,   1334,    374,   4287,    624,  15374,
           2575,    481,  13522,    510,   2575,    256,    481,   1334,     25,
            362,   2477,  83248,   1334,    315,   9069,    624,   2575,    481,
           5184,     25,    362,  26618,   4091,    914,     11,    476,   2092,
            421,    279,   1334,    374,   4287,    624,  15374,   2575,    569,
            903,   1334,    362,   2477,  83248,   1334,    315,   9069,    624,
           2575,    569,    689,    362,  26618,   4091,    914,     11,    476,
           2092,    421,    279,   1334,    374,   4287,    624,   2830,   4194,
           1691,  44491,   1857,   2490,   1334,     25,    508,    703,   2467,
           1464,    923,     30,    341,    262,    470,   1334,   7829,   1691,
            741,     92, 151645]])

## SmolLM2

In [82]:
promt = """
You are documenting a Swift codebase. Given a function and a short description of where and how it is used, write a documentation comment that helps future developers understand its purpose.
Use ///-style Swift comments.
Include parameter and return descriptions, and emphasize intent over implementation.
Return only the comment and the function.

Function:

func randomElementFromStringArray(_ array: [String]) -> String? {
    return array.randomElement()
}

### Documentation:
"""

print(model.generation_config)
inputs = tokenizer(promt, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    max_new_tokens = 256,
    temperature=0.7,         # Детерминированный ответ
    top_p=0.95,               # Нормальный сэмплинг, можно фиксировать в 0.9
    do_sample=True,          # Для генеративных моделей лучше оставить True
    eos_token_id=tokenizer.eos_token_id,  # Корректный EOS
    pad_token_id=tokenizer.eos_token_id
)

print(outputs)

GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 0
}

tensor([[  198,  2683,   359, 23672,   253, 29509,  2909,  6822,    30, 10887,
           253,  1517,   284,   253,  1890,  5410,   282,   837,   284,   638,
           357,   314,   804,    28,  2965,   253, 10822,  5189,   338,  2311,
          1774,  9994,  1044,   624,  3446,    30,   198,  9801, 13241, 29256,
          9608, 29509,  6623,    30,   198,   788,  8886,  8709,   284,  1003,
         11661,    28,   284, 11111,  7360,   690,  6230,    30,   198, 14362,
           805,   260,  5189,   284,   260,  1517,    30,   198,   198, 18226,
            42,   198,   198, 10005,  4369, 13975,  5212, 10492, 16483, 10558,
          5079,    42,   933, 10492,  2255,  4722, 16481,    47,  1662,   472,
          1003,  5079,    30,  8077, 13975,  1000,   198,   109,   198,   198,
          3757, 37140,    42,  1116,   198,  1559,    31,  9097,   253,  4369,
          4047,   429,   354,  5079,   282, 12345,    30,   198,  1559,    31,
           731, 36204,  5079,    42,   378,  5079,   288,  4491,   354,  4047,
           429,    30,   198,  1559,    31,   731,  9097,    42,   378, 11973,
          4047,    30,   198, 10005,  4369, 13975,  5212, 10492, 16483, 10558,
          5079,    42,   933, 10492,  2255,  4722, 16481,    47,  1662,   472,
          1003,  5079,    30,  8077, 13975,  1000,   198,   109,   198,   198,
            19,    35,    37,   198,   198,  3757, 17969,   198,   198, 24217,
           260,  1517,   281,   469,  2909,  6822,   338, 12413,   284,  6765,
           253,  3688,   347,   253,  4928,    30,   198,   198,  9801, 13241,
         29256,  9608, 29509,  6623,    30,   198,   788,  8886,   253,  5410,
           282,   732,   260,  1517,  1072,   284,   638,   357,   314,   804,
            30,   198,   788,  8886,   253,  5410,   282,   750,  6116,   284,
           480,  1995,    30,   198,   788,  8886,   253,  5410,   282,   260,
          1003,  1502,   284,   624,  1502, 10195,    30,   198,   198, 10912,
            42,   198,   198, 25563, 13966,  1662,   472, 16689,   288, 10492,
          1000,  4722, 16481,  1662,   448,  1003, 16481,    24,  3477,    42,
         19114,    48,  1002,   639,    25,   472,  4029,   198,   109,   198,
           198,  3757, 37140,    42,  1116,   198,  1559,    31,  9097,   253,
          4928,  6133,   282,   260,  1836,  3688,    30,   198,  1559,    31,
           731, 36204,  3688,    42,   378,  3688,   288,  4624,    30,   198,
          1559,    31,   731,  9097,    42,   378, 29199,  3688,   347,   253,
          4928,    30,   198, 10005,   288, 10492,    24,  2759,    42, 13966,
            25,  4722, 16481,  1662,   472,  1003, 16481,    24,  3477,    42,
         19114,    48,  1002,  3688,    25,   198,   109,   198,   198,    19,
            35,    38,   198,   198,  3757, 17969,   198,   198, 24217,   260]])

In [81]:
# Получаем длины исходных входов
input_lengths = inputs['input_ids'].shape[1]
# Убираем токены prompt'а
generated_tokens = outputs[:, input_lengths:]
# Декодируем только новые токены
predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

for predict in predictions:
    print("Predict: \n", predict)

Predict: 
 
/// Returns a random element from an array of strings.
///
/// - Parameter array: An array of strings.
/// - Returns: A random element from the array.
/// - Complexity: O(1)
func randomElementFromStringArray(_ array: [String]) -> String? {
    return array.randomElement()
}

### Example:

/// Returns a random element from an array of strings.
///
/// - Parameter array: An array of strings.
/// - Returns: A random element from the array.
/// - Complexity: O(1)
func randomElementFromStringArray(_ array: [String]) -> String? {
    return array.randomElement()
}

### Note:

If you want to document a function that takes no parameters, you can omit the function name and the parameters, but
you still need to document the return value.

### Example:

/// Returns a random element from an array of strings.
///
/// - Parameter array: An array of strings.
/// - Returns: A random element from the array.
/// - Complexity: O(1)

### END